In [1]:
from src.api.factor import fmp_test , perf_test
app = fmp_test(end_dt = 20240131 , prob_type = 'linprog')


In initializing PortfolioOptimizerConfig, used kwargs: {'prob_type': 'linprog'}
In initializing PortfolioOptimizerConfig, used kwargs: {'prob_type': 'linprog'}
In initializing PortfolioOptimizerConfig, used kwargs: {'prob_type': 'linprog'}
In initializing PortfolioOptimizerConfig, used kwargs: {'prob_type': 'linprog'}
In initializing PortfolioOptimizerConfig, used kwargs: {'prob_type': 'linprog'}
In initializing PortfolioOptimizerConfig, used kwargs: {'prob_type': 'linprog'}
In initializing PortfolioOptimizerConfig, used kwargs: {'prob_type': 'linprog'}
In initializing PortfolioOptimizerConfig, used kwargs: {'prob_type': 'linprog'}
In initializing PortfolioOptimizerConfig, used kwargs: {'prob_type': 'linprog'}
In initializing PortfolioOptimizerConfig, used kwargs: {'prob_type': 'linprog'}
In initializing PortfolioOptimizerConfig, used kwargs: {'prob_type': 'linprog'}
In initializing PortfolioOptimizerConfig, used kwargs: {'prob_type': 'linprog'}
Optimization of PortfolioBuilderGroup(2 

TypeError: 'NoneType' object is not callable

In [2]:
from src.api.factor import fmp_test , perf_test
app = perf_test(benchmark = None)


PerfManager calc of ic_curve Finished!
PerfManager calc of ic_decay Finished!
PerfManager calc of ic_indus Finished!
PerfManager calc of ic_year Finished!
PerfManager calc of ic_mono Finished!
PerfManager calc of pnl_curve Finished!
PerfManager calc of style_corr Finished!
PerfManager calc of grp_curve Finished!
PerfManager calc of grp_decay_ir Finished!
PerfManager calc of grp_year Finished!
PerfManager calc of distr_curve Finished!
PerfManager plot of ic_curve Finished!
PerfManager plot of ic_decay Finished!
PerfManager plot of ic_indus Finished!
PerfManager plot of ic_year Finished!
PerfManager plot of ic_mono Finished!
PerfManager plot of pnl_curve Finished!
PerfManager plot of style_corr Finished!
PerfManager plot of grp_curve Finished!
PerfManager plot of grp_decay_ir Finished!
PerfManager plot of grp_year Finished!
PerfManager plot of distr_curve Finished!
PerfManager plot Finished!
Analytic datas are saved to D:\Coding\learndl\learndl\results\perf_test\test_20241120161512\data.

In [1]:
%run src_runs/autorun/daily_update.py

Use device name: NVIDIA GeForce RTX 4090
update data: ********************
Unpack Update Files
Sat Nov 16 13:39:25 2024 : download since!
Connection and Factor preparation finished!
Sat Nov 16 13:39:25 2024 : All Updates Done! Cost 0.50 Secs
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
process other min bars:
--------------------------------------------------------------------------------
process other min bars:
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
prepare predict data: ********************
predict is True , Data Processing start!
6 datas :['y', 'day', '30m', 'style', 'indus', 'week']
y is up to 20241116 already!
day is up to 20241116 already!
30m is up to 20241116 alr

100%|██████████| 333/333 [00:00<00:00, 3737.91it/s]


--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
update_factors: ********************
Finish model [gru_day] predicting!
Finish model [gruRTN_day] predicting!
Finish model [gru_avg] predicting!
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
sending email success


In [1]:
from src.api import DataAPI , ModelAPI

#print('update_data: ' + '*' * 20)
DataAPI.update()

ModelAPI.update()


download data: ********************
DataBase object [information_ts],[calendar],[20241122] save to D:\Coding\learndl\learndl\data\DataBase\DB_information_ts\calendar.feather successfully
DataBase object [information_ts],[change_name],[20241122] save to D:\Coding\learndl\learndl\data\DataBase\DB_information_ts\change_name.feather successfully
DataBase object [information_ts],[description],[20241122] save to D:\Coding\learndl\learndl\data\DataBase\DB_information_ts\description.feather successfully
DataBase object [information_ts],[industry],[20241122] save to D:\Coding\learndl\learndl\data\DataBase\DB_information_ts\industry.feather successfully
DataBase object [benchmark_ts],[csi1000],[20241023 to 20241122]...... finished! Cost 0.00 secs
DataBase object [benchmark_ts],[csi2000],[20241023 to 20241122]...... finished! Cost 0.00 secs
DataBase object [benchmark_ts],[csi300],[20241024 to 20241122]...... finished! Cost 0.01 secs
DataBase object [benchmark_ts],[csi500],[20241023 to 20241122]..

c:\Users\jinmeng\AppData\Local\Programs\Python\Python311\Lib\site-packages\rqdatac\client.py:257: UserWarning: Your account will be expired after  13 days. Please call us at 0755-22676337 to upgrade or purchase or renew your contract.
  warnings.warn("Your account will be expired after  {} days. "


DataBase object [trade_ts],[min],[20241119] save to D:\Coding\learndl\learndl\data\DataBase\DB_trade_ts\min\2024\min.20241119.feather successfully
rcquant bar min 20241119 success
DataBase object [trade_ts],[min],[20241120] save to D:\Coding\learndl\learndl\data\DataBase\DB_trade_ts\min\2024\min.20241120.feather successfully
rcquant bar min 20241120 success
DataBase object [trade_ts],[min],[20241121] save to D:\Coding\learndl\learndl\data\DataBase\DB_trade_ts\min\2024\min.20241121.feather successfully
rcquant bar min 20241121 success
--------------------------------------------------------------------------------
process other min bars at 20241119 from source rcquant
DataBase object [trade_ts],in],[20241119] save to D:\Coding\learndl\learndl\data\DataBase\DB_trade_ts\5min\2024\5min.20241119.feather successfully
DataBase object [trade_ts],in],[20241119] save to D:\Coding\learndl\learndl\data\DataBase\DB_trade_ts\10min\2024\10min.20241119.feather successfully
DataBase object [trade_ts],i

KeyError: '[21991231] not in index'

In [1]:
from src.api import DataAPI , ModelAPI

ModelAPI.update()

prepare predict data: ********************
predict is True , Data Processing start!
6 datas : ['y', 'day', '30m', 'style', 'indus', 'week'] , from -366 to None
y is up to 20241122 already!
day blocks loading start!
 --> trade_ts blocks reading [day] DataBase...... finished! Cost 2.44 secs
day blocks loading finished! Cost 2.44 secs
day blocks process...... finished! Cost 0.38 secs
day blocks masking...... finished! Cost 0.06 secs
day blocks saving ...... finished! Cost 0.33 secs
day blocks norming...... finished! Cost 0.00 secs
day finished! Cost 3.33 Seconds
--------------------------------------------------------------------------------
30m blocks loading start!
 --> trade_ts blocks reading in] DataBase...... finished! Cost 0.43 secs
 --> trade_ts blocks reading [day] DataBase...... finished! Cost 1.76 secs
30m blocks loading finished! Cost 2.22 secs
30m blocks process...... finished! Cost 0.25 secs
30m blocks masking...... finished! Cost 0.08 secs
30m blocks saving ...... finished! 

100%|██████████| 316/316 [00:00<00:00, 2364.56it/s]


--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
update_factors: ********************


  0%|          | 0/114 [00:00<?, ?it/s]

Finish model [gru_day] predicting!


  0%|          | 0/114 [00:00<?, ?it/s]

Finish model [gruRTN_day] predicting!


  0%|          | 0/114 [00:00<?, ?it/s]

Finish model [gru_avg] predicting!
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------


In [1]:
from src.api import ModelAPI
model = ModelAPI.Testor('ts_mixer')
model.try_forward()
model.try_metrics()

D:\Coding\learndl\learndl\configs\nn\ts_mixer.yaml does not exist, trying default.yaml
Load  2 DataBlocks...... finished! Cost 0.09 secs
Align 2 DataBlocks...... finished! Cost 0.23 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


100%|██████████| 212/212 [00:00<?, ?it/s]


x shape is torch.Size([5169, 30, 6])
y shape is torch.Size([5169, 1])
Test Forward Success
x shape is torch.Size([5169, 30, 6])
y shape is torch.Size([5169, 1])
Test Forward Success
score function of [spearman] calculated and success!
loss function of [ccc] calculated and success!
metrics :  Metrics(loss=ccc,metric=spearman,penalty=['hidden_corr'])
Test Metrics Success


ModelTestor(model=NNPredictor(model_full_name=ts_mixer_day_ShortTest@0@0@0)) , check [.model][.batch_data][.metrics]

In [5]:
model.metrics.output

BatchMetric(score=0.06481771171092987, losses={'ccc': tensor([1.0014], grad_fn=<ExpBackward0>), 'hidden_corr': tensor([0.])})